[辻真吾・矢吹太朗『ゼロからはじめるデータサイエンス入門』（講談社, 2021）](https://github.com/taroyabuki/fromzero)

In [ ]:
# Google Colaboratoryの環境設定
if (file.exists("/content")) {
  options(Ncpus = parallel::detectCores())
  installed_packages <- rownames(installed.packages())
  packages_to_install <- c("caret", "ggfortify", "glmnetUtils", "leaps", "neuralnet", "psych")
  install.packages(setdiff(packages_to_install, installed_packages))
}

## 8.1 ブドウの生育条件とワインの価格

In [ ]:
library(tidyverse)
my_url <- "http://www.liquidasset.com/winedata.html"
tmp <- read.table(file = my_url,   # 読み込む対象
                  header = TRUE,   # 1行目は変数名
                  na.string = ".", # 欠損値を表す文字列
                  skip = 62,       # 読み飛ばす行数
                  nrows = 38)      # 読み込む行数
psych::describe(tmp)

In [ ]:
my_data <- na.omit(tmp[, -c(1, 2)])
head(my_data)

In [ ]:
dim(my_data)

In [ ]:
my_data %>% write_csv("wine.csv")

In [ ]:
#my_data <- read_csv("wine.csv") # 作ったファイルを使う場合
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

## 8.2 重回帰分析

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

my_model <- train(form = LPRICE2 ~ WRAIN + DEGREES + HRAIN + TIME_SV,
                  data = my_data,
                  method = "lm",
                  trControl = trainControl(method = "LOOCV"))

In [ ]:
coef(my_model$finalModel) %>%
  as.data.frame

In [ ]:
my_test <- data.frame(
  WRAIN = 500, DEGREES = 17,
  HRAIN = 120, TIME_SV = 2)
my_model %>% predict(my_test)

In [ ]:
y  <- my_data$LPRICE2
y_ <- my_model %>% predict(my_data)

RMSE(y_, y)

R2(pred = y_, obs = y,
   form = "traditional")

R2(pred = y_, obs = y,
   form = "corr")

In [ ]:
my_model$results

In [ ]:
M <- my_data[, -1] %>%
  mutate(b0 = 1) %>% as.matrix
b <- MASS::ginv(M) %*% y
matrix(b,
       dimnames = list(colnames(M)))

## 8.3 標準化

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

my_data %>%
  mutate_if(is.numeric, scale) %>% # 数値の列の標準化
  pivot_longer(-LPRICE2) %>%
  ggplot(aes(x = name, y = value)) +
  geom_boxplot() +
  stat_summary(fun = mean, geom = "point", size = 3) +
  xlab(NULL)

In [ ]:
my_model <- train(
  form = LPRICE2 ~ .,
  data = my_data,
  method = "lm",
  preProcess = c("center", "scale"))

In [ ]:
coef(my_model$finalModel) %>%
  as.data.frame

In [ ]:
my_test <- data.frame(
  WRAIN = 500, DEGREES = 17,
  HRAIN = 120, TIME_SV = 2)
my_model %>% predict(my_test)

## 8.4 入力変数の数とモデルの良さ

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

n <- nrow(my_data)
my_data2 <- my_data %>% mutate(v1 = 0:(n - 1) %% 2,
                               v2 = 0:(n - 1) %% 3)
head(my_data2)

In [ ]:
my_model2 <- train(form = LPRICE2 ~ ., data = my_data2, method = "lm",
                   trControl = trainControl(method = "LOOCV"))
y  <- my_data2$LPRICE2
y_ <- my_model2 %>% predict(my_data2)

RMSE(y_, y)

my_model2$results$RMSE

## 8.5 変数選択

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)
n <- nrow(my_data)
my_data2 <- my_data %>% mutate(v1 = 0:(n - 1) %% 2,
                               v2 = 0:(n - 1) %% 3)

In [ ]:
my_model <- train(form = LPRICE2 ~ .,
                  data = my_data2,
                  method = "leapForward", # 変数増加法
                  trControl = trainControl(method = "LOOCV"),
                  tuneGrid = data.frame(nvmax = 1:6)) # 選択する変数の上限
summary(my_model$finalModel)$outmat

## 8.6 補足：正則化

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

In [ ]:
A <- 2
B <- 0.1

my_model <- train(
  form = LPRICE2 ~ .,
  data = my_data,
  method = "glmnet",
  standardize = TRUE,
  tuneGrid = data.frame(
    lambda = A,
    alpha = B))

In [ ]:
coef(my_model$finalModel, A)

In [ ]:
my_test <- data.frame(
  WRAIN = 500, DEGREES = 17,
  HRAIN = 120, TIME_SV = 2)
my_model %>% predict(my_test)

In [ ]:
library(ggfortify)
library(glmnetUtils)

my_data2 <- my_data %>% scale %>%
  as.data.frame

B <- 0.1

glmnet(
  form = LPRICE2 ~ .,
  data = my_data2,
  alpha = B) %>%
  autoplot(xvar = "lambda") +
  xlab("log A ( = log lambda)") +
  theme(legend.position = c(0.15, 0.25))

In [ ]:
As <- seq(0, 0.1, length.out = 21)
Bs <- seq(0, 0.1, length.out =  6)

my_model <- train(
  form = LPRICE2 ~ ., data = my_data, method = "glmnet", standardize = TRUE,
  trControl = trainControl(method = "LOOCV"),
  tuneGrid = expand.grid(lambda = As, alpha  = Bs))

my_model$bestTune

In [ ]:
tmp <- "B ( = alpha)"
ggplot(my_model) +
  theme(legend.position = c(0, 1), legend.justification = c(0, 1)) +
  xlab("A ( = lambda)") +
  guides(shape = guide_legend(tmp), color = guide_legend(tmp))

In [ ]:
my_model$results %>%
  filter(RMSE == min(RMSE))

## 8.7 ニューラルネットワーク

In [ ]:
curve(1 / (1 + exp(-x)), -6, 6)

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

In [ ]:
my_model <- train(form = LPRICE2 ~ .,
                  data = my_data,
                  method = "neuralnet",              # ニューラルネットワーク
                  preProcess = c("center", "scale"), # 標準化
                  trControl = trainControl(method = "LOOCV"))
plot(my_model$finalModel) # 訓練済ネットワークの描画

In [ ]:
my_model$results

In [ ]:
my_model <- train(
  form = LPRICE2 ~ .,
  data = my_data,
  method = "neuralnet",
  preProcess = c("center", "scale"),
  trControl = trainControl(method = "LOOCV"),
  tuneGrid = expand.grid(layer1 = 1:5,
                         layer2 = 0:2,
                         layer3 = 0))

In [ ]:
my_model$results %>%
  filter(RMSE == min(RMSE))